In [ ]:
import logging

from pathlib import Path

from Pegasus.api import *

logging.basicConfig(level=logging.DEBUG)

# --- Properties ---------------------------------------------------------------
props = Properties()
props["pegasus.dir.storage.mapper"] = "Replica"
props["pegasus.dir.storage.mapper.replica"] = "YAML"
props["pegasus.dir.storage.mapper.replica.file"] = "output-rc.yml"
props.write() 

# --- Input RC -----------------------------------------------------------------
with open("f0", "w") as f:
    f.write("sample input")
    
input_rc = ReplicaCatalog()\
            .add_replica("local", "f0", Path(".").resolve() / "f0")\
            .write()

# --- Output RC ----------------------------------------------------------------
output_rc = ReplicaCatalog()\
                .add_replica("local", "f1", Path(".").resolve() / "wf-output/dir1/f1")\
                .add_replica("local", "f2", Path(".").resolve() / "wf-output/dir2/f2")\
                .write("output-rc.yml")

# --- Transformations ----------------------------------------------------------
keg = Transformation(
            "keg",
            site="condorpool",
            pfn="/home/scitech/pegasus/dist/pegasus-5.0.0dev/bin/pegasus-keg",
            is_stageable=False
        )

tc = TransformationCatalog()\
        .add_transformations(keg)\
        .write()

# --- Workflow -----------------------------------------------------------------
wf = Workflow("output-stuff")

f0 = File("f0")
f1 = File("f1")
f2 = File("f2")

job = Job("keg")\
        .add_args("-i", f0, "-o", f1, f2)\
        .add_inputs(f0)\
        .add_outputs(f1, f2, stage_out=True)

wf.add_jobs(job)

try:
    wf.plan(submit=True).wait()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.08.22 00:21:55.669 UTC:
2020.08.22 00:21:55.675 UTC:   -----------------------------------------------------------------------
2020.08.22 00:21:55.681 UTC:   File for submitting this DAG to HTCondor           : output-stuff-0.dag.condor.sub
2020.08.22 00:21:55.686 UTC:   Log of DAGMan debugging messages                 : output-stuff-0.dag.dagman.out
2020.08.22 00:21:55.692 UTC:   Log of HT

[######--------------------------------------------]  12.5% ..Running (Completed: 1, Queued: 1, Running: 0, Failed: 0)